In [124]:
# !pip install folium

In [1]:
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import requests
from dotenv import load_dotenv
import os
import time

# 사용자 좌표 (위도, 경도)
points = {
    'A': np.array([37.531132809078464, 126.8821229873256]),  # 목동운동장
    'B': np.array([37.5703884202892, 126.992329841579]),  # 종로 3가
    'C': np.array([37.513992512175925, 127.10199193245755])   # 롯데월드
}

mid_point = np.sum(list(points.values()), axis=0) / len(points)

# .env 파일에서 환경변수 로드
load_dotenv()

# 환경변수에서 API 키 불러오기
api_key = os.getenv('ODsay_APIKEY')

# ODsay API 요청 URL
url = 'https://api.odsay.com/v1/api/searchPubTransPathT'

# 허용하는 최대 소요 시간 차이 (분)
standard_time_gap = 20

# 각 유저별 (출발지 → 중간지점) 단위 벡터 저장용 딕셔너리
unit_vecs = {}

# 각 유저별 (출발지 → 중간지점) 소요 시간 가중 벡터 저장용 딕셔너리
vecs_weight = {}

# 각 유저의 초기 소요 시간을 0으로 기본 설정
defult_time = {key: 0 for key in points.keys()}

# 반복 시도 횟수 카운터
trial = 1

# 반복 루프 시작
while True:
    # 중간지점 좌표 (위도, 경도) 분리
    end_lat, end_lon = mid_point
    print(f'중간 지점 좌표: {mid_point}')

    # 각 유저별 소요 시간 계산
    for name, value in points.items():
        # 유저별 출발지 좌표
        start_lat, start_lon = points[name]

        # ODsay API 요청 파라미터 구성
        params = {
            'SX': float(start_lon),  # 출발 경도
            'SY': float(start_lat),  # 출발 위도
            'EX': float(end_lon),    # 도착 경도
            'EY': float(end_lat),    # 도착 위도
            'apiKey': api_key        # API 키
        }

        # GET 요청 전송
        response = requests.get(url, params=params)
        data = response.json()

        # 해당 유저의 첫 번째 경로 소요 시간
        short_time = data['result']['path'][0]['info']['totalTime']
        
        # 모든 경로 중 가장 짧은 시간 찾기
        for i in range(len(data['result']['path'])):
            print(f'{name}유저의 최단 시간: {short_time}')
            time_cost = data['result']['path'][i]['info']['totalTime']
            if time_cost < short_time:
                short_time = time_cost

        # 해당 유저의 최종 소요 시간을 기록
        defult_time[name] = short_time

    # 최대 소요 시간과 최소 소요 시간의 차이 계산
    time_gap = max(defult_time.values()) - min(defult_time.values())
    print(f'유저별 소요 시간: {defult_time}')
    print(f'{trial}번째 시도입니다. / {time_gap}분 차이')
    trial += 1

    # 소요 시간 차이가 허용 기준을 넘는 경우 → 중간지점 보정
    if time_gap > standard_time_gap:
        for name, point in points.items():
            # 유저 → 중간지점 벡터 계산
            vec = point - mid_point
            norm = np.linalg.norm(vec)  # 벡터의 크기 계산

            # 벡터 크기가 0이면 단위벡터 (0, 0), 아니면 정규화
            if norm == 0:
                unit_vec = np.array([0, 0])
            else:
                unit_vec = vec / norm

            # 단위벡터 저장
            unit_vecs[name] = unit_vec

        # 소요 시간 가중치를 단위벡터에 곱해서 가중 벡터 계산
        for name, vec in unit_vecs.items():
            vecs_weight[name] = vec * defult_time[name]

        # 모든 유저 가중 벡터 평균 계산
        avg_vec = sum(vecs_weight.values()) / len(points)

        # 중간지점을 평균 벡터 방향으로 조금 이동 (0.01배)
        mid_point += (avg_vec * 0.01)

        # 과도한 API 호출 방지를 위해 10초 대기
        time.sleep(10)

    # 소요 시간 차이가 기준 이하이면 중간지점 확정, 루프 종료
    else:
        print(f"중간 지점은 {mid_point}입니다")
        for name, value in defult_time.items():
            print(f'{name}의 소요 시간: {value}분')
        break

중간 지점 좌표: [ 37.53850458 126.99214825]


KeyError: 'result'

In [2]:
data

{'error': [{'code': '500',
   'message': '[ApiKeyAuthFailed] ApiKey authentication failed.'}]}

In [9]:
data['result']['path'][0]['subPath']


[{'trafficType': 3, 'distance': 452, 'sectionTime': 7},
 {'trafficType': 1,
  'distance': 2450,
  'sectionTime': 6,
  'stationCount': 4,
  'lane': [{'name': '수도권 신림선', 'subwayCode': 117, 'subwayCityCode': 1000}],
  'intervalTime': 6,
  'startName': '신림',
  'startX': 126.929643,
  'startY': 37.484732,
  'endName': '보라매',
  'endX': 126.920428,
  'endY': 37.500408,
  'way': '보라매',
  'wayCode': 1,
  'door': '0-0',
  'startID': 11718,
  'endID': 11714,
  'startExitNo': '3',
  'startExitX': 126.92961510921224,
  'startExitY': 37.48387398515152,
  'passStopList': {'stations': [{'index': 0,
     'stationID': 11718,
     'stationName': '신림',
     'x': '126.929652',
     'y': '37.484736'},
    {'index': 1,
     'stationID': 11717,
     'stationName': '당곡',
     'x': '126.927559',
     'y': '37.490253'},
    {'index': 2,
     'stationID': 11716,
     'stationName': '보라매병원',
     'x': '126.923812',
     'y': '37.49297'},
    {'index': 3,
     'stationID': 11715,
     'stationName': '보라매공원',
     '